In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from processingQst import *
from similarityFcts import *
from tqdm import tqdm
import pandas as pd

2023-05-06 14:40:15.827774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 14:40:16.220753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-06 14:40:17.258571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-06 14:40:17.275534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returni

In [2]:
df = pd.read_csv("./input/csv/train.csv",delimiter=";")

Qst = df["question"]
id  = df["id"]
nbrRepAll = df["nbr_correct_answers"]
RepListIndxAll = df["correct_answers"]
nbrRep = 5
print(len(Qst))
data_list = []
RepListIndx = 'abcde'

2171


In [3]:
def writeJson(path,data):
    with open(path,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4,ensure_ascii=False)

In [4]:
# keyword = ['bactérie','méningite']
with open('output/corpusMS.json', 'r') as f:
    corpusA = json.load(f)
with open('output/corpusMerck.json', 'r') as fs:
    corpusB = json.load(fs)

In [5]:

for cpt in range(len(Qst)):
# for cpt in tqdm(range(len(Qst))):
    if (cpt > 0):
        break
    elif (cpt>=0): 
        qst = Qst[cpt]
        # print(qst)
        nwQst = ''
        listMed = []
        keywordQ = []
        keywordQP = []
        ListMedAns = ''
        Spword = False
        data_dict = {}

        # Détécter la négation 
        if(isRequestWrongAns(qst)):
            WrongQst = True 
            # print(WrongQst)
        else: 
            WrongQst = False 
            # print(WrongQst)
        
        # supprimer les parenthèses 
        qst = del_betParenthese(qst)

        # fragementation des questions selon les ","
        fragQst = splitQst(qst)

        for icpt in range(len(fragQst)):
            if not (fragQst[icpt]==''):
                nwQst,Spword,listMed = recoverMedFrag(fragQst,icpt,nwQst,Spword)
                keywordQ.extend(listMed)
        nwQst = nwQst.lstrip()
        nwQstp = ''
        var = ''
        for j in range(5):
            keywordQP = []
            Ans = df[f"answers.{RepListIndx[j]}"][cpt] 
            Ans = Ans.lstrip()
            keywordQP.extend(RecoverListMed(Ans))
            if Spword:
                nwQstp = replaceAtFront(Ans) + nwQst + '.'
            else:
                nwQstp = nwQst + ' '+ Ans.lower() + '.' 
            # Appliquer la négation 
            if WrongQst == True:
                nwQstp = appliquer_negation(nwQstp.lower())
            
            # # test de similarité 
            # print("----")
            print(nwQstp)
            # print(keywordQ)
            # print(keywordQP)
            taux_sim = getRepDB(keywordQ,keywordQP,corpusA,nwQstp)
            taux_simB = 0# getRepDB(keywordQ,keywordQP,corpusB,nwQstp)
            if taux_sim>0.90 or taux_simB>0.90:
                if var !="":
                    var = var + '|'
                var = var + RepListIndx[j]
            
        # créer un dataframe pour les données actuelles
        data_dict = {'id': id[cpt], 'correction': var}
        data_list.append(data_dict)



In [6]:
# writeJson("output/Qst_dev.json",data_list)
df = pd.DataFrame(data_list)
df.to_csv('./output/tachePrincipale.csv', sep=';', index=False)